# Chapter 43: TypeScript Best Practices

---

## Introduction

Writing TypeScript that compiles is only the beginning. Professional TypeScript development requires adherence to conventions that ensure code is maintainable, readable, and robust across teams and time. This chapter consolidates industry-standard best practices from organizations like Microsoft, Google, Airbnb, and the TypeScript core team into actionable guidelines.

Whether you're starting a new project or refining an existing codebase, these practices will help you avoid common pitfalls, leverage TypeScript's type system fully, and create APIs that are intuitive for consumers while remaining maintainable for authors.

---

## 43.1 Code Organization

Well-organized code reduces cognitive load and makes dependencies explicit. TypeScript projects benefit from consistent structural patterns that scale from single files to monorepos.

### 43.1.1 File Structure Patterns

**Feature-Based Organization (Recommended for Applications):**

```
src/
├── features/                    # Domain-driven modules
│   ├── users/
│   │   ├── api/                 # API calls
│   │   │   ├── getUser.ts
│   │   │   └── updateUser.ts
│   │   ├── components/          # React/Vue components
│   │   │   ├── UserProfile.tsx
│   │   │   └── UserList.tsx
│   │   ├── hooks/               # Feature-specific hooks
│   │   │   └── useUser.ts
│   │   ├── stores/              # State management
│   │   │   └── userStore.ts
│   │   ├── types.ts             # Feature-specific types
│   │   ├── utils.ts             # Feature utilities
│   │   └── index.ts             # Public API
│   └── orders/
│       └── ...
├── shared/                      # Cross-cutting concerns
│   ├── components/              # Shared UI components
│   ├── hooks/                   # Shared hooks
│   ├── utils/                   # Shared utilities
│   └── types/                   # Global types
├── lib/                         # Third-party configurations
│   ├── api.ts                   # Axios/fetch setup
│   └── db.ts                    # Database client
└── main.ts                      # Application entry
```

**Explanation:**
- Feature-based organization groups related code by domain (users, orders) rather than technical role (components, utils)
- Co-location reduces navigation overhead—everything for a feature is in one place
- `index.ts` files (barrel exports) define the public API of each feature
- `shared/` contains truly global utilities used across multiple features

**Layer-Based Organization (Recommended for Libraries):**

```
src/
├── types/                       # Public type definitions
│   ├── index.ts
│   └── api.ts
├── utils/                       # Pure functions
│   ├── validation.ts
│   └── formatting.ts
├── services/                    # Business logic
│   └── userService.ts
├── adapters/                    # External integrations
│   └── databaseAdapter.ts
└── index.ts                     # Library entry point
```

### 43.1.2 Barrel Exports (Index Files)

Barrel files centralize exports, creating clean public APIs for modules.

**Implementation:**

```typescript
// features/users/index.ts
// Export only what consumers need - hide implementation details

// Types
export type { User, UserRole, UserPreferences } from './types';
export type { CreateUserInput, UpdateUserInput } from './schemas';

// Components
export { UserProfile } from './components/UserProfile';
export { UserList } from './components/UserList';
export { UserCard } from './components/UserCard';

// Hooks
export { useUser } from './hooks/useUser';
export { useUsers } from './hooks/useUsers';

// Utilities (if public)
export { formatUserName } from './utils/formatUserName';

// Do NOT export internal helpers
// export { internalHelper } from './utils/internalHelper'; // Keep private
```

**Explanation:**
- Barrel files act as the "public API" for a module
- Consumers import from the feature root: `import { UserProfile } from './features/users'`
- Internal implementation can be refactored without breaking imports
- Prevents deep import chains: `../../../features/users/components/UserProfile` becomes `./features/users`

**Anti-Pattern to Avoid:**

```typescript
// ❌ Avoid wildcard exports that expose internals
export * from './types';
export * from './components';
export * from './utils';
export * from './api';
// This exposes implementation details and makes refactoring dangerous
```

### 43.1.3 Import Path Organization

Group imports logically for readability:

```typescript
// 1. External dependencies (frameworks, libraries)
import React from 'react';
import { useQuery } from '@tanstack/react-query';
import { z } from 'zod';

// 2. Absolute internal imports (aliases)
import { Button } from '@/components/ui/Button';
import { useAuth } from '@/hooks/useAuth';
import type { User } from '@/types';

// 3. Relative imports (sibling files)
import { UserAvatar } from './UserAvatar';
import { formatUserDate } from './utils';
import type { UserCardProps } from './types';

// 4. CSS/assets (if not handled by bundler)
import './UserCard.css';
```

**Explanation:**
- External dependencies change least frequently—place at top
- Absolute aliases (`@/`) indicate cross-module dependencies
- Relative imports indicate tight coupling to current directory
- Type-only imports should be grouped or marked with `type` keyword

---

## 43.2 Naming Conventions

Consistent naming reduces ambiguity and leverages IDE autocomplete effectively.

### 43.2.1 Casing Conventions by Type

```typescript
// PascalCase: Classes, Interfaces, Types, Enums, Namespaces
class UserManager { }
interface UserConfig { }
type UserID = string;
enum UserRole { Admin, User }
namespace UserUtils { }

// camelCase: Variables, Functions, Methods, Properties
const currentUser = getUser();
function fetchUser() { }
const userConfig = { maxRetries: 3 };

// UPPER_SNAKE_CASE: Constants, Enum members
const MAX_RETRY_COUNT = 3;
const API_BASE_URL = 'https://api.example.com';
enum Status { ACTIVE = 'ACTIVE', INACTIVE = 'INACTIVE' }

// camelCase or kebab-case: File names (be consistent)
// userProfile.ts or user-profile.ts
// UserCard.tsx (PascalCase for React components)
```

### 43.2.2 Interface vs Type Alias Naming

**Industry Standard (No I-prefix):**

```typescript
// ✅ Modern standard - no prefix
interface User {
  id: string;
  name: string;
}

interface UserRepository {
  findById(id: string): Promise<User>;
}

// ❌ Legacy style - avoid I-prefix
interface IUser { }  // Not recommended in modern TypeScript
interface IUserRepository { }
```

**Explanation:**
- Hungarian notation (I-prefix) originated in C# and early TypeScript
- Modern TypeScript uses structural typing—`interface` vs `type` is implementation detail
- IDEs clearly indicate types vs values through syntax highlighting
- Consistency with JavaScript ecosystem (no interfaces there anyway)

**Distinguishing Types from Values:**

```typescript
// When you need both a type and constructor/value with same name
// Use casing to distinguish (idiomatic TypeScript)

// Type (PascalCase)
interface Config {
  apiUrl: string;
}

// Constructor function/value (camelCase)
function createConfig(apiUrl: string): Config {
  return { apiUrl };
}

// Or namespace merging (advanced)
namespace Config {
  export const DEFAULT_API_URL = 'https://api.example.com';
}

// Usage
const config: Config = createConfig(Config.DEFAULT_API_URL);
```

### 43.2.3 Generic Parameter Naming

```typescript
// Single letter for simple, obvious generics
function identity<T>(value: T): T {
  return value;
}

// Descriptive names for complex generics
interface ApiResponse<Data, ErrorCode extends string> {
  data: Data;
  error?: {
    code: ErrorCode;
    message: string;
  };
}

// Common conventions:
// T = Type (general)
// K = Key
// V = Value
// E = Element (arrays)
// P = Props (React)
// S = State

// Multiple type parameters
interface Map<K, V> {
  get(key: K): V | undefined;
  set(key: K, value: V): void;
}

// Constrained generics (descriptive names help)
interface Repository<Entity extends { id: string }, CreateInput> {
  findById(id: string): Promise<Entity>;
  create(input: CreateInput): Promise<Entity>;
}
```

### 43.2.4 Boolean Naming

Booleans should read as predicates:

```typescript
// ✅ Predicate style (is/has/should/can)
isActive
hasPermission
shouldRetry
canDelete
isLoading
hasError

// ❌ Ambiguous nouns
status  // string? boolean?
loading // verb, not state
error   // Error object? boolean?
valid   // valid what?

// ✅ For React props
interface ButtonProps {
  isDisabled?: boolean;
  isLoading?: boolean;
  shouldFullWidth?: boolean;
}

// ✅ For configuration
interface Config {
  isCacheEnabled: boolean;
  hasStrictMode: boolean;
  shouldLogErrors: boolean;
}
```

---

## 43.3 Type vs Interface Guidelines

TypeScript offers two ways to define object shapes: `interface` and `type`. While often interchangeable, they have subtle differences that inform when to use each.

### 43.3.1 Key Differences

```typescript
// 1. Declaration Merging (Interface only)
interface User {
  name: string;
}

interface User {
  age: number;  // Merged with above - User now has name and age
}

// Type aliases cannot be merged
type Person = { name: string };
// type Person = { age: number }; // Error: Duplicate identifier

// 2. Extending (Both work differently)
interface Animal {
  name: string;
}

interface Dog extends Animal {  // extends keyword
  breed: string;
}

type Cat = Animal & {  // intersection operator
  color: string;
};

// 3. Unions (Type only)
type Status = 'active' | 'inactive' | 'pending';
type Response = Success | Error;

// interface StatusUnion = 'active' | 'inactive'; // Error

// 4. Mapped types (Type only)
type ReadonlyUser = {
  readonly [K in keyof User]: User[K];
};

// 5. Performance (Historically significant, now minimal)
// Interfaces used to be faster for complex hierarchies
// Modern TypeScript optimizes both similarly
```

### 43.3.2 Decision Matrix

```typescript
┌─────────────────────────────────────────────────────────────────────┐
│                    Interface vs Type Decision Guide                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Use INTERFACE when:                                               │
│   ┌─────────────────────────────────────────────────────────────┐ │
│   │ • Defining object shapes that may need extension            │ │
│   │ • Creating public API definitions (libraries)               │ │
│   │ • Implementing classes (class implements Interface)           │ │
│   │ • Declaration merging is needed (e.g., augmenting third     │ │
│   │   party types)                                                │ │
│   │ • Preferring explicit error messages for excess properties    │ │
│   └─────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Use TYPE when:                                                    │
│   ┌─────────────────────────────────────────────────────────────┐ │
│   │ • Defining unions (string | number)                           │ │
│   │ • Using mapped types (Partial, Pick, etc.)                  │ │
│   │ • Creating tuple types ([string, number])                   │ │
│   │ • Defining function types                                     │ │
│   │ • Using conditional types                                   │ │
│   │ • Needing to alias primitive types (type ID = string)        │ │
│   │ • Working with complex type transformations                   │ │
│   └─────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Either is fine:                                                   │
│   • Simple object shapes in application code                          │
│   • Consistency within a project is more important than choice      │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 43.3.3 Consistency Rules

**Rule 1: Be Consistent Within Projects**

```typescript
// If your team prefers interfaces for objects, use them consistently
interface UserConfig { }
interface DatabaseConfig { }
interface ApiConfig { }

// Don't mix arbitrarily:
interface UserConfig { }
type DatabaseConfig = { };  // Inconsistent style
```

**Rule 2: Library Public APIs**

```typescript
// Libraries should prefer interfaces for public-facing types
// This allows consumers to augment types via declaration merging

// @mycompany/types package
export interface User {
  id: string;
  name: string;
}

// Consumer can augment:
declare module '@mycompany/types' {
  interface User {
    customField: string;  // Augmentation works with interface
  }
}
```

**Rule 3: Complex Types**

```typescript
// Use type for complex transformations
type DeepPartial<T> = {
  [P in keyof T]?: T[P] extends object ? DeepPartial<T[P]> : T[P];
};

type EventPayload<T extends string> = T extends 'user.created' 
  ? { userId: string } 
  : T extends 'order.completed' 
  ? { orderId: string; amount: number }
  : never;

// Use interface for straightforward object definitions
interface UserService {
  getUser(id: string): Promise<User>;
  saveUser(user: User): Promise<void>;
}
```

---

## 43.4 Avoiding `any`

The `any` type disables TypeScript's type checking. While occasionally necessary, it should be treated as a code smell and minimized through strict typing strategies.

### 43.4.1 The Cost of `any`

```typescript
// ❌ any disables all type safety
function processData(data: any): any {
  return data.user.name;  // No error even if data is null
}

// Runtime error possible, but no compile-time warning
processData(null);  // Runtime: Cannot read property 'user' of null
processData({});  // Runtime: Cannot read property 'name' of undefined

// ❌ any is contagious
const result: any = fetchData();
const upper = result.toUpperCase();  // No error, might not exist
const fixed = result.toFixed(2);     // No error, might be string

// ❌ any breaks autocomplete and refactoring
// IDE cannot provide intellisense for 'any' typed variables
// Renaming properties won't update 'any' references
```

### 43.4.2 Alternatives to `any`

**Use `unknown` for External Data:**

```typescript
// ✅ unknown forces type checking before use
function processData(data: unknown): string {
  // Type guard required
  if (typeof data === 'string') {
    return data.toUpperCase();  // OK, narrowed to string
  }
  
  if (isUser(data)) {  // Type predicate
    return data.name;
  }
  
  throw new Error('Invalid data format');
}

// Type guard function
function isUser(value: unknown): value is User {
  return (
    typeof value === 'object' &&
    value !== null &&
    'id' in value &&
    'name' in value &&
    typeof (value as User).id === 'string' &&
    typeof (value as User).name === 'string'
  );
}
```

**Use Generics for Flexible APIs:**

```typescript
// ❌ Using any loses type information
function fetchAny(url: string): Promise<any> {
  return fetch(url).then(r => r.json());
}

// ✅ Generic preserves return type
function fetchJSON<T>(url: string): Promise<T> {
  return fetch(url).then(r => r.json());
}

// Usage with explicit type
const user = await fetchJSON<User>('/api/user');
// user is typed as User, full autocomplete available
```

**Use `Record` for Dynamic Objects:**

```typescript
// ❌ any for dynamic keys
function processConfig(config: any): void {
  Object.keys(config).forEach(key => {
    console.log(config[key]);  // No type safety
  });
}

// ✅ Record for known value types
function processConfig(config: Record<string, string>): void {
  Object.entries(config).forEach(([key, value]) => {
    console.log(value.toUpperCase());  // OK, value is string
  });
}

// ✅ Index signatures for specific patterns
interfaceTranslations {
  [locale: string]: {
    greeting: string;
    farewell: string;
  };
}
```

### 43.4.3 Gradual Migration Strategies

**1. Type Assertions with Comments:**

```typescript
// Temporary escape hatch with documentation
function legacyFunction(): any {
  // Returns untyped data from old API
}

// Mark with TODO and type assertion
const data = legacyFunction() as User;  // TODO: Remove when API is typed
```

**2. Strict TypeScript Configuration:**

```typescript
// tsconfig.json
{
  "compilerOptions": {
    "strict": true,           // Master switch for strictness
    "noImplicitAny": true,    // Error on implicit any
    "strictNullChecks": true, // Catch null/undefined errors
    "noImplicitThis": true,   // Catch 'this' context errors
    "alwaysStrict": true      // Strict mode JavaScript
  }
}
```

**3. Linting with ESLint:**

```typescript
// .eslintrc.json
{
  "rules": {
    "@typescript-eslint/no-explicit-any": "error",
    "@typescript-eslint/no-unsafe-argument": "error",
    "@typescript-eslint/no-unsafe-assignment": "error",
    "@typescript-eslint/no-unsafe-call": "error",
    "@typescript-eslint/no-unsafe-member-access": "error",
    "@typescript-eslint/no-unsafe-return": "error"
  }
}
```

**4. TypeScript-Strict-Plugin for Legacy Code:**

```typescript
// For gradual migration, use strict-comments
// tsconfig.strict.json extends base but adds strictness incrementally

// In code:
// @ts-strict-ignore
function legacyUntypedFunction(data: any) {
  // This function is exempt from strict checking during migration
}
```

---

## 43.5 Strict Mode Recommendations

TypeScript's strict mode enables all strict type-checking options. For new projects, it should always be enabled. For existing projects, it provides a target for gradual migration.

### 43.5.1 Strict Mode Configuration

```typescript
// tsconfig.json - The gold standard
{
  "compilerOptions": {
    "strict": true,  // Enables all strict options (recommended)
    
    // Or enable individually:
    "noImplicitAny": true,
    "noImplicitThis": true,
    "strictNullChecks": true,
    "strictFunctionTypes": true,
    "strictPropertyInitialization": true,
    "strictBindCallApply": true,
    "alwaysStrict": true,
    "useUnknownInCatchVariables": true,
    "exactOptionalPropertyTypes": true,  // Strict optional handling
    "noUncheckedIndexedAccess": true,    // Strict index access
    "noImplicitReturns": true,
    "noFallthroughCasesInSwitch": true,
    "noUncheckedSideEffectImports": true
  }
}
```

### 43.5.2 Strict Null Checks Deep Dive

`strictNullChecks` is the most impactful strict option, preventing null/undefined errors.

```typescript
// Without strictNullChecks (unsafe):
function getLength(str: string | null): number {
  return str.length;  // No error, runtime crash possible
}

// With strictNullChecks (safe):
function getLengthSafe(str: string | null): number {
  // Error: Object is possibly 'null'
  // return str.length;
  
  // Must handle null case:
  if (str === null) {
    return 0;
  }
  return str.length;  // OK, narrowed to string
}

// Optional properties become strict
interface User {
  name: string;
  nickname?: string;
}

function displayUser(user: User): void {
  console.log(user.name);
  
  // Error under strictNullChecks (without exactOptionalPropertyTypes):
  // console.log(user.nickname.toUpperCase());
  
  // Must check:
  if (user.nickname) {
    console.log(user.nickname.toUpperCase());
  }
}
```

### 43.5.3 Exact Optional Property Types

TypeScript 4.4+ adds stricter optional property handling:

```typescript
// Without exactOptionalPropertyTypes:
interface Config {
  timeout?: number;
}

const config: Config = {
  timeout: undefined  // Allowed, but confusing
};

// With exactOptionalPropertyTypes:
interface StrictConfig {
  timeout?: number;
}

const strictConfig: StrictConfig = {
  timeout: undefined  // Error: Type 'undefined' is not assignable
};

// Must actually omit the property:
const correctConfig: StrictConfig = {};  // OK
const explicitConfig: StrictConfig = {
  timeout: 5000  // OK
};
```

### 43.5.4 No Unchecked Indexed Access

Prevents unsafe array/object index access:

```typescript
// Without noUncheckedIndexedAccess:
function getFirst(items: string[]): string {
  return items[0];  // Returns string, might be undefined
}

// With noUncheckedIndexedAccess:
function getFirstSafe(items: string[]): string | undefined {
  return items[0];  // Type is string | undefined
}

// Must handle undefined:
function processItems(items: string[]): void {
  const first = items[0];
  if (first) {
    console.log(first.toUpperCase());  // OK
  }
  
  // Or use non-null assertion when sure (rarely):
  console.log(items[0]!.toUpperCase());  // ! asserts non-null
}
```

---

## 43.6 Error Handling Patterns

TypeScript enables typed error handling, moving beyond try/catch with unknown errors to explicit, type-safe error representations.

### 43.6.1 The Problem with Throwing

```typescript
// ❌ JavaScript throwing is untyped
async function fetchUser(id: string): Promise<User> {
  const response = await fetch(`/api/users/${id}`);
  
  if (!response.ok) {
    throw new Error('Failed to fetch');  // Could be anything
  }
  
  return response.json();
}

// Consumer has no idea what errors might occur
try {
  const user = await fetchUser('123');
} catch (error) {
  // error is unknown in strict mode
  // Could be Error, string, number, or anything thrown elsewhere
}
```

### 43.6.2 Result Type Pattern (Rust/FP Inspired)

```typescript
// Define Result type
type Result<T, E = Error> = 
  | { success: true; data: T }
  | { success: false; error: E };

// Implementation
async function fetchUserSafe(id: string): Promise<Result<User, ApiError>> {
  try {
    const response = await fetch(`/api/users/${id}`);
    
    if (!response.ok) {
      const errorData = await response.json();
      return {
        success: false,
        error: {
          code: errorData.code || 'UNKNOWN',
          message: errorData.message || 'Request failed',
          status: response.status
        }
      };
    }
    
    const data = await response.json();
    return { success: true, data };
    
  } catch (e) {
    return {
      success: false,
      error: {
        code: 'NETWORK_ERROR',
        message: e instanceof Error ? e.message : 'Unknown error',
        status: 0
      }
    };
  }
}

// Usage requires handling both cases
const result = await fetchUserSafe('123');

if (!result.success) {
  // TypeScript knows result.error exists
  console.error(result.error.message);
  return;
}

// TypeScript knows result.data exists
console.log(result.data.name);
```

### 43.6.3 Discriminated Union Errors

```typescript
// Specific error types for different failure modes
type UserError = 
  | { type: 'USER_NOT_FOUND'; userId: string }
  | { type: 'VALIDATION_ERROR'; fields: Record<string, string> }
  | { type: 'NETWORK_ERROR'; retryable: boolean }
  | { type: 'PERMISSION_DENIED'; required: string[] };

async function getUser(id: string): Promise<Result<User, UserError>> {
  try {
    const response = await fetch(`/api/users/${id}`);
    
    if (response.status === 404) {
      return {
        success: false,
        error: { type: 'USER_NOT_FOUND', userId: id }
      };
    }
    
    if (response.status === 403) {
      return {
        success: false,
        error: { type: 'PERMISSION_DENIED', required: ['admin'] }
      };
    }
    
    // ... handle other cases
    
    const user = await response.json();
    return { success: true, data: user };
    
  } catch {
    return {
      success: false,
      error: { type: 'NETWORK_ERROR', retryable: true }
    };
  }
}

// Exhaustive error handling
const result = await getUser('123');

if (!result.success) {
  switch (result.error.type) {
    case 'USER_NOT_FOUND':
      showNotFound(result.error.userId);
      break;
    case 'VALIDATION_ERROR':
      showValidationErrors(result.error.fields);
      break;
    case 'NETWORK_ERROR':
      if (result.error.retryable) retry();
      break;
    case 'PERMISSION_DENIED':
      redirectToLogin();
      break;
    default:
      // TypeScript ensures all cases handled
      const _exhaustive: never = result.error;
  }
}
```

### 43.6.4 Custom Error Classes

When throwing is necessary, use typed custom errors:

```typescript
// Base error class
class AppError extends Error {
  constructor(
    message: string,
    public code: string,
    public statusCode: number
  ) {
    super(message);
    this.name = this.constructor.name;
    Error.captureStackTrace(this, this.constructor);
  }
}

// Specific error types
class ValidationError extends AppError {
  constructor(
    message: string,
    public fields: Record<string, string[]>
  ) {
    super(message, 'VALIDATION_ERROR', 400);
  }
}

class NotFoundError extends AppError {
  constructor(resource: string, id: string) {
    super(`${resource} with id ${id} not found`, 'NOT_FOUND', 404);
  }
}

// Type guard for error handling
function isAppError(error: unknown): error is AppError {
  return error instanceof AppError;
}

// Usage
try {
  await processUser(data);
} catch (error) {
  if (isAppError(error)) {
    // TypeScript knows error.code, error.statusCode exist
    if (error.statusCode === 404) {
      showNotFound();
    } else if (error instanceof ValidationError) {
      showErrors(error.fields);
    }
  } else {
    // Unknown error
    console.error('Unexpected error:', error);
  }
}
```

---

## 43.7 Documentation with JSDoc/TSDoc

TypeScript's IntelliSense is enhanced by TSDoc comments, providing hover information, parameter descriptions, and usage examples directly in the IDE.

### 43.7.1 TSDoc Basics

```typescript
/**
 * Calculates the area of a rectangle.
 * 
 * @param width - The width of the rectangle in pixels
 * @param height - The height of the rectangle in pixels
 * @returns The calculated area in square pixels
 * 
 * @example
 * ```typescript
 * const area = calculateArea(10, 20);
 * console.log(area); // 200
 * ```
 */
function calculateArea(width: number, height: number): number {
  return width * height;
}

/**
 * Configuration options for the API client.
 */
interface ApiClientConfig {
  /**
   * The base URL for all API requests.
   * Must include protocol (https://) and no trailing slash.
   */
  baseUrl: string;
  
  /**
   * Request timeout in milliseconds.
   * @defaultValue 5000
   */
  timeout?: number;
  
  /**
   * Maximum number of retry attempts for failed requests.
   * @defaultValue 3
   */
  maxRetries?: number;
}
```

### 43.7.2 Advanced TSDoc Tags

```typescript
/**
 * User management service.
 * 
 * @remarks
 * This service handles CRUD operations for users. All methods
 * return promises and should be awaited.
 * 
 * @example
 * Basic usage:
 * ```typescript
 * const service = new UserService();
 * const user = await service.findById('123');
 * ```
 * 
 * @see {@link UserRepository} for database operations
 * @see {@link https://docs.example.com/users|User Documentation}
 */
class UserService {
  /**
   * Finds a user by their unique identifier.
   * 
   * @param id - The UUID of the user to find
   * @returns A promise that resolves to the user, or null if not found
   * 
   * @throws {NotFoundError} When the user does not exist
   * @throws {ValidationError} When the ID format is invalid
   * 
   * @deprecated Use {@link findByIdSafe} instead which returns Result type
   */
  async findById(id: string): Promise<User | null> {
    // Implementation
  }
  
  /**
   * Type-safe version with error handling.
   * 
   * @param id - The UUID of the user
   * @returns Result containing user or error details
   */
  async findByIdSafe(id: string): Promise<Result<User, UserError>> {
    // Implementation
  }
}

/**
 * Supported user roles in the system.
 */
enum UserRole {
  /** Full system access including user management */
  Admin = 'admin',
  
  /** Standard application access */
  User = 'user',
  
  /** Read-only access for viewing content */
  Guest = 'guest'
}
```

### 43.7.3 Type-Only Imports in Documentation

```typescript
/**
 * @packageDocumentation
 * Core types for the application domain.
 */

// Use import types for documentation links without runtime imports
import type { User } from './user';
import type { Order } from './order';

/**
 * Creates a relationship between a user and an order.
 * 
 * @param user - The {@link User} to associate
 * @param order - The {@link Order} to link
 * @returns The association record
 */
function createAssociation(user: User, order: Order): Association {
  return { userId: user.id, orderId: order.id };
}
```

### 43.7.4 Markdown Support

```typescript
/**
 * Processes user data with advanced transformations.
 * 
 * ## Features
 * - Validates email format
 * - Normalizes names (title case)
 * - Checks for duplicate entries
 * 
 * ## Usage Notes
 * > **Warning:** This function modifies the input object in place.
 * > Consider using {@link immutableProcess} for functional approach.
 * 
 * ## Example
 * ```typescript
 * const user = { name: 'john doe', email: 'JOHN@EXAMPLE.COM' };
 * processUser(user);
 * // user is now { name: 'John Doe', email: 'john@example.com' }
 * ```
 * 
 * @param user - The user object to process
 * @returns The processed user (same reference)
 */
function processUser(user: User): User {
  // Implementation
}
```

---

## 43.8 Chapter Summary and Exercises

### Chapter Summary

This chapter established standards for professional TypeScript development:

1. **Code Organization**: Feature-based folder structures, barrel exports for clean APIs, and logical import grouping improve maintainability at scale.

2. **Naming Conventions**: PascalCase for types/classes, camelCase for values, predicate naming for booleans (is/has/should), and avoiding Hungarian notation (I-prefix).

3. **Type vs Interface**: Interfaces for extensible public APIs and declaration merging; types for unions, mapped types, and complex transformations. Consistency within projects trumps theoretical purity.

4. **Avoiding Any**: Use `unknown` for external data, generics for flexible APIs, and `Record` for dynamic objects. Enable strict linting rules and migrate legacy code gradually.

5. **Strict Mode**: Enable `strict: true` for new projects. Key options include `strictNullChecks` (null safety), `exactOptionalPropertyTypes` (strict optionals), and `noUncheckedIndexedAccess` (array safety).

6. **Error Handling**: Prefer Result types over throwing for explicit error handling. Use discriminated unions for typed errors. When throwing is necessary, use custom error classes with type guards.

7. **Documentation**: TSDoc comments enhance IDE IntelliSense with descriptions, examples, deprecation notices, and cross-references.

### Best Practices Checklist

```
┌─────────────────────────────────────────────────────────────────────┐
│                    TypeScript Best Practices Checklist               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Code Style                                                        │
│   □ Use strict: true in tsconfig.json                               │
│   □ No explicit any (use unknown or generics)                       │
│   □ Consistent naming (PascalCase types, camelCase values)          │
│   □ No I-prefix on interfaces                                       │
│   □ Boolean predicates (is/has/should)                              │
│                                                                     │
│   Architecture                                                      │
│   □ Feature-based folder structure                                  │
│   □ Barrel exports (index.ts) for module APIs                       │
│   □ Group imports: external → absolute → relative                   │
│   □ Private implementation details not exported                     │
│                                                                     │
│   Type Safety                                                       │
│   □ strictNullChecks enabled                                        │
│   □ Runtime validation at system boundaries (Zod/io-ts)             │
│   □ Type guards for unknown narrowing                               │
│   □ Discriminated unions for complex states                         │
│                                                                     │
│   Error Handling                                                    │
│   □ Result types for fallible operations                            │
│   □ Custom error classes with type guards                           │
│   □ Exhaustive error case handling                                  │
│                                                                     │
│   Documentation                                                     │
│   □ TSDoc for public APIs                                           │
│   □ Examples in complex function docs                                 │
│   □ @deprecated tags with alternatives                              │
│   □ Cross-references with @see and {@link}                          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### Practical Exercises

**Exercise 1: Refactoring to Strict Types**

Refactor this code to eliminate `any` and follow strict mode:

```typescript
// Before
function processData(data: any): any {
  return data.items.map(item => ({
    id: item.id,
    name: item.name.toUpperCase()
  }));
}

// After: Use unknown, type guards, and proper typing
// Requirements:
// 1. Input should be unknown
// 2. Validate data structure with type guard
// 3. Return typed array
// 4. Handle edge cases (null items, missing properties)
```

**Exercise 2: API Design**

Design a type-safe API client following these requirements:

```typescript
// Create a fetch wrapper that:
// 1. Uses generics for request/response types
// 2. Returns Result type instead of throwing
// 3. Has typed error cases (NetworkError, HttpError, ParseError)
// 4. Supports request/response interceptors with proper typing
// 5. Includes TSDoc documentation with examples

interface ApiClient {
  get<T>(url: string): Promise<Result<T, ApiError>>;
  post<T, B>(url: string, body: B): Promise<Result<T, ApiError>>;
  // ... etc
}
```

**Exercise 3: Error Handling Refactor**

Convert a try/catch-based API to Result types:

```typescript
// Current implementation throws
class UserService {
  async getUser(id: string): Promise<User> {
    if (!id) throw new Error('Invalid ID');
    const user = await db.find(id);
    if (!user) throw new NotFoundError('User not found');
    return user;
  }
}

// Refactor to:
// 1. Define UserError union type with specific error cases
// 2. Return Result<User, UserError>
// 3. Update all callers to handle errors exhaustively
// 4. Ensure no exceptions escape (wrap unexpected errors)
```

**Exercise 4: Module Architecture**

Organize a complex feature:

```typescript
// Create a "Payment" feature with:
// 1. Folder structure following feature-based organization
// 2. Barrel exports defining public API
// 3. Internal utilities not exported
// 4. Types defined as interfaces (extensible) vs types (unions)
// 5. Strict error handling for payment processing
// 6. Full TSDoc documentation

// Should include:
// - Payment methods (credit card, paypal, crypto)
// - Validation schemas
// - API integration
// - React hooks (if applicable)
// - Utility functions
```

**Exercise 5: Strict Mode Migration**

Given a legacy codebase with these settings:

```typescript
// tsconfig.json (current)
{
  "compilerOptions": {
    "strict": false,
    "noImplicitAny": false,
    "strictNullChecks": false
  }
}

// Create a migration plan:
// 1. Enable strict flags one by one
// 2. For each flag, identify common code patterns that break
// 3. Provide automated refactoring scripts (if possible)
// 4. Create lint rules to prevent regression
// 5. Document the migration timeline for a 50-file codebase
```

---

## Next Chapter Preview

### Chapter 44: Common Pitfalls and How to Avoid Them

In the next chapter, we will examine the most frequent mistakes TypeScript developers encounter and provide strategies to avoid them:

- **Type vs Value Confusion**: Understanding when TypeScript erases types and runtime value expectations
- **Incorrect Type Assertions**: When `as` is appropriate vs. dangerous
- **Overusing Type Assertions**: Relying on assertions instead of proper type guards
- **Ignoring Null/Undefined**: The billion-dollar mistake in TypeScript
- **Mutable vs Immutable Confusion**: Readonly arrays, const assertions, and mutation bugs
- **Generic Misuse**: Over-constraining or under-constraining type parameters
- **Circular Dependencies**: Module resolution issues and architectural solutions

This chapter will serve as a debugging guide and anti-pattern reference to help you recognize and resolve common TypeScript issues before they become production bugs.

---

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../12. typescript_ecosystem/42. typescript_at_scale.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='44. common_pitfalls_and_how_to_avoid_them.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
